In [13]:
import gdal
import os
import csv
import pandas as pd
import math
import numpy as np
import collections

In [2]:
path = 'D://datasets//major//5552743//' #master folder path
csv_path='D://datasets//major//5552743//changed_excel_file.csv' #path of csv file

folder1 = 'IndianVillagesDataset_30m//imagery_res30_48bands//'
folder2 = 'IndianVillagesDataset_30m//masks_res30//'

dirs1 = os.listdir(path+folder1) #images from folder1
dirs2 = os.listdir(path+folder2) #images from folder2


In [3]:
dirs1[:6]

['Bihar(vil)-1029000103080400-246652.tif',
 'Bihar(vil)-Aadharpur-227831.tif',
 'Bihar(vil)-Ababakarpur-235044.tif',
 'Bihar(vil)-Ababakarpur-235152.tif',
 'Bihar(vil)-Ababakarpur-236676.tif',
 'Bihar(vil)-Abad Milik-224942.tif']

In [4]:
dataset = {}
csv_dict = {}

low_limit = 12
upper_limit = 23

In [52]:

def calc_ndvi(image, mask, low_limit, upper_limit, size):
    band_percs = []
    
    for i in range(low_limit,upper_limit + 1):

        temp = image[i-1]*mask
        temp1 = np.where(np.logical_and(temp>0.2,temp<=0.6))
        perc = len(temp1[0])/size
        band_percs.append(perc)

    return sum(band_percs)/len(band_percs)

In [65]:
def calc_evi_log(image, mask, low_limit, upper_limit, size):
    band_percs = []
    
    for i in range(low_limit,upper_limit + 1):
        if str(image[i-1][0][0]) != "nan":
            temp = image[i-1]*mask

            sumb = np.sum(temp)




            band_percs.append(math.log(sumb))

    return sum(band_percs)/len(band_percs)

In [63]:
def calc_evi_mean(image, mask, low_limit, upper_limit, size):
    band_percs = []
    
    for i in range(low_limit,upper_limit + 1):
        if str(image[i-1][0][0]) != "nan":

            temp = image[i-1]*mask

            mean = np.mean(temp)


            band_percs.append(mean)

    return sum(band_percs)/len(band_percs)

In [64]:
def calc_evi_median(image, mask, low_limit, upper_limit, size):
    band_percs = []
    
    for i in range(low_limit,upper_limit + 1):
        if str(image[i-1][0][0]) != "nan":
            temp = image[i-1]*mask

            median = np.median(temp)


            band_percs.append(median)

    return sum(band_percs)/len(band_percs)

In [21]:
def infra_density(num_o_houses, mask):
    area = (np.count_nonzero(mask))*10
    den = num_o_houses/area
    return den

In [11]:
csv = pd.read_csv(csv_path)
csv.head(10)
x = csv.sort_values(by=['Village Name','Census 2011 ID'])
x.head(10)

C:\Users\UB\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Census 2011 ID,Village Name,District Name,State Name,Percentage,Electrified,Number of Households
21555,246652.0,1.03E+15,Bhojpur,Bihar,57.3,1,452.0
23528,246188.0,ALawalpur,Patna,Bihar,70.3,1,1032.0
13389,227831.0,Aadharpur,Darbhanga,Bihar,22.8,1,889.0
15303,235044.0,Ababakarpur,Vaishali,Bihar,85,1,240.0
16274,235152.0,Ababakarpur,Vaishali,Bihar,58.4,1,997.0
16598,236676.0,Ababakarpur,Samastipur,Bihar,18.5,1,195.0
23561,224942.0,Abad Milik,Katihar,Bihar,0,0,-9.0
22782,224943.0,Abadpur,Katihar,Bihar,39,1,1931.0
21233,246649.0,Abatana,Bhojpur,Bihar,96.7,1,30.0
16980,235346.0,Abbu Chak,Vaishali,Bihar,87.5,1,8.0


In [12]:
status = x.drop([23528],axis=0)
status.head(10)
status['Electrified'][0]

'0'

In [74]:
index = 0

village_name = list()
ndvi =list()
evi_log = list()
electrified = list()
evi_mean = list()
evi_median = list()
infra_den = list()

for file in dirs1[index:10000]:
    raster1 = gdal.Open(path+folder1+file)
    arr1 = raster1.ReadAsArray()

    raster2 = gdal.Open(path+folder2+file)
    arr2 = raster2.ReadAsArray()
    arr2 = arr2/255
    
    h = arr2.shape[0]
    w = arr2.shape[1]
    size = h*w
    
    village_name.append(file)
    ndvi.append(calc_ndvi(arr1, arr2, 12, 23, size))
    evi_log.append(calc_evi_log(arr1, arr2, 24, 35, size))
    evi_mean.append(calc_evi_mean(arr1, arr2, 24, 35, size))
    evi_median.append(calc_evi_median(arr1, arr2, 24, 35, size))
    electrified.append(status['Electrified'][index])
    infra_den.append(infra_density(abs(status['Number of Households'][index]),arr2))
    
    index+=1
print("donee")
    
dataset["village_name"] = village_name

dataset["ndvi"] = ndvi

dataset["evi_log"] = evi_log

dataset["evi_mean"] = evi_mean

dataset["evi_median"] = evi_median

dataset["infra_density"] = infra_den

dataset["electrified"] = electrified
    
    

# dataset is the final dict that contains ndvi info of all ndvi bands as well as electrification status

    


C:\Users\UB\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in greater
  import sys
C:\Users\UB\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in less_equal
  import sys


donee


In [73]:
type(dirs1)

list

In [75]:
tt = pd.DataFrame.from_dict(dataset)

In [76]:
tt.shape

(10000, 7)

In [77]:
tt = tt[['village_name', 'ndvi', 'evi_log', 'evi_mean', 'evi_median',
       'infra_density', 'electrified']]

In [78]:
tt.head(20)

,village_name,ndvi,evi_log,evi_mean,evi_median,infra_density,electrified
0,Bihar(vil)-1029000103080400-246652.tif,0.426946,6.683821,1.344866,1.559850,0.001793,0
1,Bihar(vil)-Aadharpur-227831.tif,0.428579,9.350204,1.136851,1.522555,0.006673,1
2,Bihar(vil)-Ababakarpur-235044.tif,0.417025,7.605374,1.112016,1.626774,0.029530,1
3,Bihar(vil)-Ababakarpur-235152.tif,0.364170,9.025839,0.931878,1.362547,0.000189,0
4,Bihar(vil)-Ababakarpur-236676.tif,0.501645,7.565639,1.013988,1.533472,0.000819,0
5,Bihar(vil)-Abad Milik-224942.tif,0.332933,6.622085,0.929262,0.000000,0.027157,1
6,Bihar(vil)-Abadpur-224943.tif,0.364810,10.229015,0.969755,0.000000,0.000064,0
7,Bihar(vil)-Abatana-246649.tif,0.314316,7.723282,0.990071,1.489608,0.000657,0
8,Bihar(vil)-Abbu Chak-235346.tif,0.523333,6.331983,1.351320,1.641588,0.002679,0
9,Bihar(vil)-Abbupur-238177.tif,0.427805,6.423491,1.018820,1.540599,0.039628,1


In [80]:
tt.to_csv(path+"database.csv", index=False)